In [1]:
#######
IN_DATA_FNAMES = ['/oak/stanford/orgs/kipac/users/delon/LensQuEst/map_sims_%d.pkl'%(i) for i in range(1,51)]
import warnings
warnings.filterwarnings("ignore")
#####

In [2]:
import os, sys
WORKING_DIR = os.path.dirname(os.path.abspath(''))
sys.path.insert(1, os.path.join(WORKING_DIR,'LensQuEst'))

In [3]:
from universe import *
from halo_fit import *
from cmb import *
from flat_map import *
from weight import *
from pn_2d import *
import pickle
import seaborn as sns
from scipy.stats import spearmanr
import matplotlib
from tqdm import trange, tqdm

In [4]:
print("Map properties")

# number of pixels for the flat map
nX = 1200
nY =1200

# map dimensions in degrees
sizeX = 20.
sizeY = 20.

# basic map object
baseMap = FlatMap(nX=nX, nY=nY, sizeX=sizeX*np.pi/180., sizeY=sizeY*np.pi/180.)

# multipoles to include in the lensing reconstruction
lMin = 30.; lMax = 3.5e3

# ell bins for power spectra
nBins = 51  # number of bins
lRange = (1., 2.*lMax)  # range for power spectra

Map properties


In [5]:
oup_fname = '../data/input/universe_Planck15/camb/CAMB_outputs.pkl'
print(oup_fname)
f = open(oup_fname, 'rb') 
powers,cl,c_lensed,c_lens_response = pickle.load(f)
f.close()

totCL=powers['total']
unlensedCL=powers['unlensed_scalar']

L = np.arange(unlensedCL.shape[0])

unlensedTT = unlensedCL[:,0]/(L*(L+1))*2*np.pi
F = unlensedTT
funlensedTT = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)

L = np.arange(cl.shape[0])
PP = cl[:,0]
rawPP = PP*2*np.pi/((L*(L+1))**2)
rawKK = L**4/4 * rawPP

fKK = interp1d(L, rawKK, kind='linear', bounds_error=False, fill_value=0.)

L = np.arange(totCL.shape[0])

lensedTT = totCL[:,0]/(L*(L+1))*2*np.pi
F = lensedTT
flensedTT = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)


ftot = lambda l : flensedTT(l) + cmb.fForeground(l) + cmb.fdetectorNoise(l)


L = np.arange(c_lens_response.shape[0])

cTgradT = c_lens_response.T[0]/(L*(L+1))*2*np.pi

fTgradT = interp1d(L, cTgradT, kind='linear', bounds_error=False, fill_value=0.)

../data/input/universe_Planck15/camb/CAMB_outputs.pkl


In [6]:
# Adjust the lMin and lMax to the assumptions of the analysis
# CMB S4/SO specs
cmb = StageIVCMB(beam=1.4, noise=7., lMin=lMin, lMaxT=lMax, lMaxP=lMax, atm=False)

# Total power spectrum, for the lens reconstruction
# basiscally gets what we theoretically expect the
# power spectrum will look like
forCtotal = lambda l: ftot(l) 

# reinterpolate: gain factor 10 in speed
L = np.logspace(np.log10(lMin/2.), np.log10(2.*lMax), 1001, 10.)
F = np.array(list(map(forCtotal, L)))
cmb.fCtotal = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)

In [7]:
print("Gets a theoretical prediction for the noise")
fNqCmb_fft = baseMap.forecastN0Kappa(fTgradT, cmb.fCtotal, lMin=lMin, lMax=lMax, test=False)
Ntheory = lambda l: fNqCmb_fft(l) 

Gets a theoretical prediction for the noise
computing the reconstruction noise


In [8]:
# In[12]:
in_data = {}
for file_idx in trange(1,51):
    fname = IN_DATA_FNAMES[file_idx-1]
    f = open(fname, 'rb') 
    c_in_data = pickle.load(f) 
    f.close()
    for key in c_in_data:
        if(key not in in_data.keys()):
            in_data[key] = np.array(c_in_data[key])
        else:
            in_data[key] = np.vstack( (in_data[key],np.array(c_in_data[key])) )


for key in in_data:
    print(key, np.shape(in_data[key]))

  0%|          | 0/50 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/oak/stanford/orgs/kipac/users/delon/LensQuEst/map_sims_1.pkl'

In [ ]:
ps_data = {}


#estimate GRF Nhat
ck = 'GRF'
for data_idx in trange(500):
    curr_data = in_data['totalF_0'][data_idx]
    
    c_ps_data = {}
    c_ps_data[ck] = [0,0,0]
    c_ps_data[ck][0], c_ps_data[ck][1], c_ps_data[ck][2] = baseMap.powerSpectrum(dataFourier=curr_data, nBins=nBins)
    if(ck not in ps_data.keys()):
        ps_data[ck] = np.array([c_ps_data[ck]])
    else:
        ps_data[ck] = np.vstack(( ps_data[ck], np.array([c_ps_data[ck]])))  

In [ ]:
ck = 'lensed'
for data_idx in trange(500):
    curr_data = in_data['totalF_1'][data_idx]
    
    c_ps_data = {}
    c_ps_data[ck] = [0,0,0]
    c_ps_data[ck][0], c_ps_data[ck][1], c_ps_data[ck][2] = baseMap.powerSpectrum(dataFourier=curr_data, nBins=nBins)
    if(ck not in ps_data.keys()):
        ps_data[ck] = np.array([c_ps_data[ck]])
    else:
        ps_data[ck] = np.vstack(( ps_data[ck], np.array([c_ps_data[ck]])))  

In [ ]:
L, ClExpected= baseMap.binTheoryPowerSpectrum(ftot, nBins=nBins)

In [ ]:
def combine_Cl(Cls_tot):
    n_runs = np.shape(Cls_tot)[0]
    print(n_runs, np.shape(Cls_tot))
    lCen = Cls_tot[0][0]
    Cls = np.sum(np.transpose(Cls_tot, axes=[1,2,0])[1], axis=1)/n_runs
#     sCls = np.sqrt(np.sum(np.square(np.transpose(Cls_tot, axes=[1,2,0])[2]), axis=1))/n_runs
    sCls = np.std(np.transpose(Cls_tot, axes=[1,2,0])[1], axis=1)/np.sqrt(n_runs)
    return lCen, Cls, sCls

In [ ]:
ps_data['GRF'].shape
fig = plt.figure(figsize=(16,9))
axs=[fig.add_axes((0.2,0.2,.75,.6)), fig.add_axes((0.2,0.0,.75,.2)),  fig.add_axes((0.2,-0.2,.75,.2))]
plt.rcParams['text.usetex'] = True

plt.rcParams['font.size'] = 20



    
lCen, Cl, sCl = combine_Cl(ps_data['GRF'])
Ipos = np.where(Cl>0)
axs[0].errorbar(lCen[Ipos], (Cl[Ipos]), yerr=sCl[Ipos], alpha=.75, 
                fmt='-', capsize=3, capthick=1, label='PS of GRF data', c='blue')

assert((lCen[Ipos] == L[Ipos]).all())

axs[1].errorbar(lCen[Ipos], (Cl[Ipos] - ClExpected[Ipos])/ClExpected[Ipos], yerr=sCl[Ipos]/ClExpected[Ipos], alpha=.75, 
                fmt='-', capsize=3, capthick=1, label='GRF data', c='blue')


theoryIpos = np.where(ClExpected > 0)
axs[0].plot(L[theoryIpos], ClExpected[theoryIpos], 'red', label=r'$C_L^{\rm tot}$ [Binned]')    


unbinnedCl = np.array(list(map(ftot, L)))
axs[0].plot(L[theoryIpos], unbinnedCl[theoryIpos], 'green', label=r'$C_L^{\rm tot}$ [Unbinned]')    

for lCen, Cl, sCl in ps_data['GRF']:
    Ipos = np.where(Cl>0)
    axs[0].plot(lCen[Ipos], Cl[Ipos], alpha=0.005, color='blue')
    assert((lCen[Ipos] == L[Ipos]).all())
    axs[1].plot(lCen[Ipos], (Cl[Ipos] - ClExpected[Ipos])/ClExpected[Ipos], alpha=0.01, color='blue')
    axs[2].plot(lCen[Ipos], (Cl[Ipos] - unbinnedCl[Ipos])/unbinnedCl[Ipos], alpha=0.01, color='blue')

# axs[0].plot(lCen[Ipos], Cl[Ipos], alpha=1, color='blue', label='GRFs')

axs[0].legend(frameon=False)
axs[0].set_xscale('log')
axs[0].set_yscale('log')
axs[0].set_xlim(lCen[Ipos][0], lCen[Ipos][-1])
axs[1].set_xlim(lCen[Ipos][0], lCen[Ipos][-1])
axs[2].set_xlim(lCen[Ipos][0], lCen[Ipos][-1])

axs[1].set_xscale('log')
# axs[1].set_yscale('log')

# axs[1].fill_between([0, 1e20], [-0.001, -0.001], [0.001, 0.001], alpha=1, color='0.85')#, label=r'$<1\%$ Error')

axs[1].axhline(0, c='red')

axs[1].set_ylim(-.98e-2, .98e-2)




axs[2].set_xscale('log')
axs[2].axhline(0, c='green')
axs[2].set_ylim(-.98e-1, .98e-1)

axs[1].set_ylabel('Binned \nFrac. \nResidual')
axs[2].set_ylabel('Un-binned \nFrac. \nResidual')
axs[2].set_xlabel(r'$L$')


In [ ]:
# UH OH !!!!

In [ ]:
ps_data['lensed'].shape
fig = plt.figure(figsize=(16,9))
axs=[fig.add_axes((0.2,0.2,.75,.6)), fig.add_axes((0.2,0.0,.75,.2)),  fig.add_axes((0.2,-0.2,.75,.2))]
plt.rcParams['text.usetex'] = True

plt.rcParams['font.size'] = 20



    
lCen, Cl, sCl = combine_Cl(ps_data['lensed'])
Ipos = np.where(Cl>0)
axs[0].errorbar(lCen[Ipos], (Cl[Ipos]), yerr=sCl[Ipos], alpha=.75, 
                fmt='-', capsize=3, capthick=1, label='PS of lensed data', c='blue')

assert((lCen[Ipos] == L[Ipos]).all())

axs[1].errorbar(lCen[Ipos], (Cl[Ipos] - ClExpected[Ipos])/ClExpected[Ipos], yerr=sCl[Ipos]/ClExpected[Ipos], alpha=.75, 
                fmt='-', capsize=3, capthick=1, label='lensed data', c='blue')


theoryIpos = np.where(ClExpected > 0)
axs[0].plot(L[theoryIpos], ClExpected[theoryIpos], 'red', label=r'$C_L^{\rm tot}$ [Binned]')    


unbinnedCl = np.array(list(map(ftot, L)))
axs[0].plot(L[theoryIpos], unbinnedCl[theoryIpos], 'green', label=r'$C_L^{\rm tot}$ [Unbinned]')    

for lCen, Cl, sCl in ps_data['lensed']:
    Ipos = np.where(Cl>0)
    axs[0].plot(lCen[Ipos], Cl[Ipos], alpha=0.005, color='blue')
    assert((lCen[Ipos] == L[Ipos]).all())
    axs[1].plot(lCen[Ipos], (Cl[Ipos] - ClExpected[Ipos])/ClExpected[Ipos], alpha=0.01, color='blue')
    axs[2].plot(lCen[Ipos], (Cl[Ipos] - unbinnedCl[Ipos])/unbinnedCl[Ipos], alpha=0.01, color='blue')


axs[0].legend(frameon=False)
axs[0].set_xscale('log')
axs[0].set_yscale('log')
axs[0].set_xlim(lCen[Ipos][0], lCen[Ipos][-1])
axs[1].set_xlim(lCen[Ipos][0], lCen[Ipos][-1])
axs[2].set_xlim(lCen[Ipos][0], lCen[Ipos][-1])

axs[1].set_xscale('log')
# axs[1].set_yscale('log')

# axs[1].fill_between([0, 1e20], [-0.001, -0.001], [0.001, 0.001], alpha=1, color='0.85')#, label=r'$<1\%$ Error')

axs[1].axhline(0, c='red')

axs[1].set_ylim(-.98e-2, .98e-2)




axs[2].set_xscale('log')
axs[2].axhline(0, c='green')
axs[2].set_ylim(-.98e-1, .98e-1)

axs[1].set_ylabel('Binned \nFrac. \nResidual')
axs[2].set_ylabel('Un-binned \nFrac. \nResidual')
axs[2].set_xlabel(r'$L$')
